#Subclassing and Superclassing: Multimodal SoftMax

Our SoftMax class's `poly` attribute allows us to input a polygon, from which it automatically generates all the normal vectors and turns them into weights (following the process in [Chapter 2](02_from_templates.ipynb)). But, can we specify *any* polygon? 

As we've seen in the T-shape example, we need to ensure the sum of all the normals is zero. We fixed that by subdividing a class into multiple classes that ensure the sum of all normals is zero, but we manually subdivided the South class to do so. Is there an algorithm we can develop to subdivide our classes into subclasses that sum to zero, then recombine those subclasses into the original classes we wanted? Would we be asking that question if we didn't have an answer?

## Superclasses and Multimodal SoftMax

### Example: T-shape revisited

## Subclassing through convex decomposition

### Example: Camera model

In [1]:
from IPython.core.display import HTML

# Borrowed style from Probabilistic Programming and Bayesian Methods for Hackers
def css_styling():
    styles = open("../styles/custom.css", "r").read()
    return HTML(styles)
css_styling()